# NIHCXR Synthetic Drift - Categorical Shift

## Load Libraries

In [ ]:
from cyclops.monitor import (
    Detector,
    Experimenter,
    Reductor,
    SyntheticShiftApplicator,
    TSTester,
)
from cyclops.monitor.plotter import plot_drift_samples_pval
from cyclops.monitor.utils import Loader

## Query Data

In [ ]:
import os

import pandas as pd
from datasets import Dataset, Image

from cyclops.monitor.utils import nihcxr_preprocess

In [ ]:
nihcxr_dir = "~/NIHCXR"
df = pd.read_csv(os.path.join(nihcxr_dir, "Data_Entry_2017.csv"))
df = nihcxr_preprocess(df, nihcxr_dir)
nih_ds = Dataset.from_pandas(df, preserve_index=False)
nih_ds = nih_ds.cast_column("image", Image())

## Initalize Reductor, Tester & Detector

In [ ]:
reductor = Reductor(
    dr_method="TAE_txrv_CNN",
)

tester = TSTester(
    tester_method="mmd",
)


detector = Detector(
    reductor=reductor,
    tester=tester,
)
with Loader("Initializing the detector..."):
    detector.fit(nih_ds, progress=False)

## Setup Baseline Experiment

In [ ]:
baseline_experiment = Experimenter(
    "sensitivity_test",
    detector=detector,
)

## Setup Drift Experiments (Categorical Shift)

In [ ]:
shiftapplicators = []
shift_type = ["categorical_shift"] * 3
cat_col = ["gender", "view", "age"]
target_categories = ["M", "PA", "18-35"]

for s_type, col, target in zip(shift_type, cat_col, target_categories):
    shiftapplicators.append(
        SyntheticShiftApplicator(
            shift_type=s_type,
            categorical_column=col,
            target_category=target,
        )
    )

experiments = []
for shiftapplicator in shiftapplicators:
    drift_experiment = Experimenter(
        "sensitivity_test",
        detector=detector,
        shiftapplicator=shiftapplicator,
    )
    experiments.append(drift_experiment)

## Run Experiments

In [ ]:
baseline_results = baseline_experiment.run(nih_ds)
drift_results = []
for experiment in experiments:
    drift_results.append(experiment.run(nih_ds))

## Gather Results

In [ ]:
results_dict = {}
results_dict.update({"baseline": baseline_results})
for itr, result in enumerate(drift_results):
    results_dict.update({f"{cat_col[itr]}: {target_categories[itr]}": result})

## Plot Experimental Results

In [ ]:
plot_drift_samples_pval(results_dict, 0.05)